In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
df = pd.read_csv('new_big_fbgm.csv')
#df = pd.read_csv('FBGM_League_1_all_seasons_Average_Stats(1).csv')
#df = df.sample(10000)
#['QB', 'OL', 'TE', 'DL', 'S', 'CB', 'WR', 'LB', 'RB', 'P', 'K']
clf2 = None
unique_pos = df.Pos.unique()
df.shape

In [ ]:
#orig_df = df.copy()
#df = orig_df.sample(frac=0.1)
#df.shape

In [ ]:
stats_to_pred = (df.columns[105:])
stats_to_use = ['Cmp',
 'Att',
 'Pct',
 'Yds',
 'TD',
 'Int',
 'Y/A',
 'AY/A',
 'QBRat',
 'Rush',
 'Yds.2',
 'TD.1',
 'Rec',
 'Yds.3',
 'TD.2',
 'PR',
 'Yds.8',
 'TD.5',
 'KR',
 'Yds.9',
 'TD.6']

In [ ]:
need_norm = ['Cmp', 'Att', 'Yds', 'TD', 'Int', 'Sk', 'Yds.1','Rush', 'Yds.2', 'TD.1','Tgt', 'Rec', 'Yds.3', 'TD.2', 'Tch','RRTD',
 'Int.1',
 'Yds.4',
 'TD.3',
 'PD',
 'FF',
 'FR',
 'Yds.5',
 'TD.4',
 'Sk.1',
 'Tck',
 'Solo',
 'Ast',
 'TFL',
 'Sfty',
 'Fmb',
 'FG10',
 'FGA10',
 'FG20',
 'FGA20',
 'FG30',
 'FGA30',
 'FG40',
 'FGA40',
 'FG50',
 'FGA50',
 'FGM',
 'FGA',
 'XPM',
 'XPA',
 'Pnt',
 'Yds.7',
 'PR',
 'Yds.8',
 'TD.5',
 'KR',
 'Yds.9',
 'TD.6',
 'FL',
 'Pen',
 'Yds.6',
 'Blk',
'YScm',
 'APY',
 'AV']
need_norm = list(set(need_norm))

In [ ]:
BAD_SET = set(['Blk','FL', 'Pen', 'Yds.6','Lng.4', 'APY'])
need_norm = [_ for _ in need_norm if _ not in BAD_SET]

In [ ]:
if False:
    for col in need_norm:
        df[col + '_PerG'] = df[col]/np.maximum(df['G'],1)
        stats_to_use.append(col + '_PerG')
    for col in need_norm:
        del stats_to_use[stats_to_use.index(col)]

In [ ]:
stats_to_use = [_ for _  in stats_to_use if _ not in BAD_SET]

In [ ]:
pos_t = 'QB'
stat_t = 'ThP'
#stat_t_idx = list(y.columns).index(stat_t)

In [ ]:
replacement_player_mean_bs = {}
replacement_player_std_bs = {}
replacement_player_cov_bs = {}
replacement_player_mean_r = {}
replacement_player_std_r = {}
replacement_player_cov_r = {}

clfs = {}
scalersX = {}
scalersY = {}
fexps = {}
for pos in unique_pos:
    y = df[df.Pos == pos]#.iloc[:,105:]
    X = df[df.Pos == pos]#.iloc[:,11:103]
    y = y[stats_to_pred]
    
    valid_col = stats_to_use#[_ for _ in X.columns if not _ in ['Blk','FL', 'Pen', 'Yds.6','Lng.4', 'APY']]
    X = X[valid_col]
    replacement_filter = (df.Pos == pos) & (df.Salary >= 0.10) & (df.Salary < 1.0)
    replacement_player_mean_bs[pos] = X[replacement_filter].mean()
    replacement_player_std_bs[pos] = X[replacement_filter].std()
    replacement_player_cov_bs[pos] = X[replacement_filter].cov()

    replacement_player_mean_r[pos] = y[replacement_filter].mean()
    replacement_player_std_r[pos] = y[replacement_filter].std()
    replacement_player_cov_r[pos] = y[replacement_filter].cov()
    
    fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=True)
    scalerX = preprocessing.StandardScaler()
    scalery = preprocessing.StandardScaler()
    prescale_X = scalerX.fit_transform(X)
    prescale_y = scalery.fit_transform(y)
    prescale_X = fexp.fit_transform(prescale_X)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(prescale_X, prescale_y, test_size=0.25, random_state=42)
    
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNet(alpha=5e-2))
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    print(pos,clf.score(prescale_X,prescale_y))
    clfs[pos] = clf
    scalersX[pos] = scalersX
    scalersY[pos] = scalery
    fexps[pos] = fexp
    
    if False and pos == pos_t:
        fscores = sklearn.feature_selection.f_regression(prescale_X,prescale_y[:,stat_t_idx])[1]
        for i in np.argsort(fscores):
            if np.isnan(fscores[i]) or fscores[i] > 0.1:
                continue
            print(X.columns[i],fscores[i])

In [ ]:
def get_norms(X):
    means = np.zeros(X.shape[1])
    stds  = np.zeros(X.shape[1])
    for col in range(X.shape[1]):
        xv = np.array(X)[:,col]
        #print(xv[np.where(xv !=0)].shape,X.columns[col])
        means[col] = xv[np.where(xv !=0)].mean()
        stds[col] = xv[np.where(xv !=0)].std()
    rX = (np.array(X)-means)/stds
    rX[np.where(X == 0)] = -10
    return rX,means,stds

In [ ]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

pos = 'QB'
stat_t = 'ThV'
stat_t_idx = list(y.columns).index(stat_t)
for i,c in enumerate(y.columns):
    coeffs = clfs[pos].estimators_[i].coef_ 
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v[:10]]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Variable | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:25s}|{:.5f}|'.format(n,v))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))



In [ ]:
GEN_YEAR = 2018
tyear = [GEN_YEAR]
#if tyear[0] < 1980:
#    print("MY PARSING OF THE TABLES IS WRONG WITHOUT the 2PA/3PA TRACKS")
#    raise

#CURRENT_YEAR = 2019
all_tables = {}
for ty in tyear: 
    all_tables[ty] = np.load('cfb_tables_{}.pkl'.format(ty))
teams = all_tables[tyear[0]].keys()
print(all_tables[GEN_YEAR]['alabama'].keys())

In [ ]:
import sklearn.feature_selection

In [ ]:
all_tables[GEN_YEAR]['alabama']['roster']

In [ ]:
player_stats = {k:{} for k in tyear}
table_columns = defaultdict(dict)
for ty in tyear:
    tables = all_tables[ty]
    for team in tables:
        team_tables =  tables[team]
        for table_name in team_tables:
            if table_name in ['draft-rights','team_and_opponent','conf','name','logo']:
                continue
            table = team_tables[table_name].fillna(0)

            #print(table_name)
            #print(table.index)
            for row in table.itertuples():
                name = row[0]
                name = name.replace('\xa0\xa0',' ').replace('*','').replace('+','')
                if name == 'Team Totals':
                    continue
                nsplit = name.split(' ')
                if nsplit[-1] in ['Jr.','Sr.','I','II','III',"IV",'(TW)']:
                    name = ' '.join(nsplit[:-1])

                player_table = player_stats[ty].get(name,{})
                player_row = player_table.get(table_name,[])
                player_row = player_row + [row]

                player_table[table_name] = player_row
                player_table['team'] = team
                player_stats[ty][name] = player_table
                
                #if name == 'Dennis Smith Jr.' or name == 'Luka Doncic':
                #    print(player_stats[ty][name],team)
            tcs = list(table.columns)
            for i in range(len(tcs)):
                while tcs.index(tcs[i]) != i:
                    if '.' in tcs[i]:
                        tsplit = tcs[i].split('.')
                        tsplit[-1] = str(int(tsplit[-1]) + 1)
                        tcs[i] = '.'.join(tsplit)
                    else:
                        tcs[i] += '.1'
            table_columns[(ty,team)][table_name] = tcs


In [ ]:
#player_stats[2007].keys()

In [ ]:
table_mask = defaultdict(dict)
for key in table_columns:
    for table in table_columns[key]:
        table_mask[key][table] = [_.strip() !=''  for _ in table_columns[key][table] ]
        table_columns[key][table] = [_ for _ in table_columns[key][table] if _.strip() != '']
#for player in player_stats:
#    for table_in in player_stats[player]:
#        if 'on_off' in table_in or 'salaries' in table_in:
#            continue
#        if len(player_stats[player][table_in]) > 1:
#            pass
            #print(player,table_in,'MP' in player_stats[player][table_in][0]._fields)
            #print(player_stats[player][table_in][0])

In [ ]:
# add playoff data to normal data
if False:
    for ty in tyear:
        for player in player_stats[ty]:
            for table_in in player_stats[ty][player]:
                tableN = table_in.split('_')
                tableS = '_'.join(tableN[1:])
                if 'playoffs'==tableN[0] and not table_in in ['playoffs_pbp']:
                    #print(table_in)
                    if tableS in player_stats[ty][player]:
                        player_stats[ty][player][tableS] += player_stats[ty][player][table_in]

In [ ]:
#player_stats[1968]['Bill Allen']
#player_stats[1968]['Jim Burns']['per_game'], table_columns[(1968,'DLC')]['per_game'].index('MP')

In [ ]:
for ty in tyear:
    for player in player_stats[ty]:
        team = player_stats[ty][player]['team']
        for tt in player_stats[ty][player]:
            if tt == 'team':
                continue
            new_rows = []
            for tablet in player_stats[ty][player][tt]:
                vector = [_ if _ != '' else '0.0' for _ in tablet[1:]]
                vector = [(float(_.replace('%',''))/100 if type(_) == str and'%' in _ else _) for _ in vector]
                if 'on_off' in tt:
                    vector = vector[1:]
                if 'contracts' in tt:
                    vector = vector[1:-2]
                if tt in ['salaries2','contracts']:
                    vector = [_.replace(',','').replace('$','') for _ in vector]
                try:
                    v2 = np.array(vector).astype(np.float)
                except:
                    v2 = vector
                new_rows.append(vector)
            a = np.array(new_rows)


            if tt in table_columns[(ty,team)]:
                if 'MP' in table_columns[(ty,team)][tt] and not tt in ['pbp','on_off','on_off_p']:
                    try:
                        a = a.astype(np.float)
                    except:
                        a = list(a)
                        a[0] = np.array([float(_) for _ in a[0]])
                        a[1] = np.array([float(_) for _ in a[1]])
                        a = np.array(a)
                    try:
                        mins = a[:,table_columns[(ty,team)][tt].index('MP')].reshape((-1,1))
                        new_rows = ((a.T @ mins)/mins.sum()).T
                        a = new_rows
                    except:
                        #print(tt,a.shape,player,a,mins)
                        #print('.',end='')
                        
                        print(ty,team,player,tt)
                        continue
            player_stats[ty][player][tt] = a


In [ ]:
player_stats

In [ ]:
len(X.columns)

In [ ]:
X.columns

In [ ]:
#'returns', 'kicking'
#table_columns[(2018,'nwe')]['kicking']


In [ ]:
lt = [('Class','roster'),('Pos','roster'),
      ('Cmp','passing'),('Att','passing'),('Pct','passing','Pct'),
      ('Yds','passing'),('TD','passing'),('Int','passing'),
      ('Y/A','passing'),('AY/A','passing'),('QBRat','passing','Rate'),
      ('Rush','rushing_and_receiving','Att'),
      ('Yds.2','rushing_and_receiving','Yds'),('TD.1','rushing_and_receiving','TD'),
      ('Rec','rushing_and_receiving'),('Yds.3','rushing_and_receiving','Yds.1'),
      ('TD.2','rushing_and_receiving','TD.1'),('Int.1','defense','Int'),
      ('Yds.4','defense','Yds'),('TD.3','defense','TD'),('PD','defense'),
      ('FF','defense'),('FR','defense'),('Yds.5','defense','Yds.1'),('TD.4','defense','TD.1'),('Sk.1','defense','Sk'),
      ('Tck','defense','Comb'),('Solo','defense'),('Ast','defense'),('TFL','defense'),('Sfty','defense'),('Fmb','defense'),
      ('FGM','kicking','FGM'),('FGA','kicking','FGA'),
      ('Pct.1','kicking','FG%'),('XPM','kicking'),('XPA','kicking'),('Pct.2','kicking','XP%'),('Pnt','kicking'),
      ('Yds.7','kicking','Yds'),('Blk','kicking','Blck'),('Y/A.2','kicking','Y/P'),
      ('PR','returns','Ret'),('Yds.8','returns','Yds'),('TD.5','returns','TD'),
      ('KR','returns','Ret.1'),('Yds.9','returns','Yds.1'),('TD.6','returns','TD.1')      ]

locations = []
for loc in lt:
    if len(loc) == 2:
        loc = list(loc)
        loc.append(loc[0])
    locations.append(loc)

        
locs = defaultdict(dict)
for ty in tyear:
    for team in all_tables[ty]:
        for loc in locations:
            try:
                locs[(ty,team)][loc[0]] = (table_columns[(ty,team)][loc[1]].index(loc[2]),loc[1])
            except Exception as e:
                #print(e)
                #print(team,ty)
                print(e,loc)
                pass#print(loc,table_columns[loc[1]])
    #if 'shooting' in table_columns:
    #    l = list(locs['LowPostFGP'])
    #    l[0] +=2
    #    locs['MidRangeFGP'] = l
len(locs[(GEN_YEAR,'alabama')]),locs[(GEN_YEAR,'alabama')]

In [ ]:
player_stats[2018]['Isaiah Sanders']['roster']

In [ ]:
player_vectors = []
player_names = []
player_years = []
player_scales = []
player_heights = []
player_weights = []
player_drafts = []
player_ages = []
player_teams = []
player_pos = []
for ty in tyear:
    for name in player_stats[ty]:
        try:
            stats = player_stats[ty][name]
            team = stats['team']

            d = {}
            for k,v in locs[(ty,team)].items():
                SV  = 0
                try:
                    SV = str(stats[v[1]][0][v[0]])
                    SV = float(SV)
                except:
                    pass
                d[k] = 0 if SV == '' else SV
            ht = 72
            wt = 200
            #print(d)
            exp = {'FR':0,'SO':1,'JR':2,'SR':3, 0.0:0}[d['Class']]
            #for col in need_norm:
            #    if col in d:
            #        d[col + '_PerG'] = d[col]/np.maximum(d.get('G',0),1)
            player_vectors.append([d[stat] for stat in X.columns])
            player_names.append(name)
            player_years.append(ty)
            player_ages.append(exp+18)
            player_pos.append([d['Pos']])
            player_heights.append(ht)
            player_drafts.append(exp)
            player_weights.append(wt)

            player_teams.append(team)
        except Exception as e:
            print(name,team)
            print(e)
            print(d.keys())
            #if name == 'Bill Bradley':
            #    raise
            raise
X.columns

In [ ]:
player_stats[ty][name]

In [ ]:
first_n = len([yr for yr in player_years if yr == tyear[0]])
gen_FA = 0#len(teams)*5 if CURRENT_YEAR!=GEN_YEAR else 0
first_n,len(teams),gen_FA
#player_names

In [ ]:
Xn = np.nan_to_num(np.array(player_vectors))
#Xn.shape,Xn_s.shape,prescale_X.shape,fexp.n_input_features_,X.shape
#clf2 = None
Xn.shape

In [ ]:
if clf2 is None:
    clf2 = linear_model.LogisticRegression()
    scalerX_pos_og = preprocessing.StandardScaler()

    fexp_pos = preprocessing.PolynomialFeatures(degree=1,interaction_only=False)
    #tmpX = df.iloc[:,11:103]
    tmpX = df[valid_col]

    clf2.fit(fexp_pos.fit_transform(scalerX_pos_og.fit_transform(tmpX)),df.Pos)

scalerX_pos = preprocessing.StandardScaler()
Xn_scale = fexp_pos.transform(scalerX_pos.fit_transform(Xn))
pred_pos = clf2.predict(Xn_scale)
    #print(clf2.score(Xn_scale,df.Pos))

In [ ]:
'QB', 'OL', 'TE', 'DL', 'S', 'CB', 'WR', 'LB', 'RB', 'P', 'K'
pos_mapping = {
 'LG':'OL',
 'RG':'OL',
 'CB':'CB',
 'LOLB':'LB',
 'RB':'RB',
 'FB':'RB',
 'K':'K',
 'C':'OL',
 'LCB':'CB',
 'RCB':'CB',
 'FS':'S',
 'RDT':'DL',
 'QB':'QB',
 'DL':'DL',
 'MLB':'LB',
 'WR':'WR',
 'S':'S',
 'SS':'S',
 'TE':'TE',
 'DT':'DL',
 'RILB':'LB',
 'T':'OL',
 'LDT':'DL',
 'P':'P',
 'LILB':'LB',
 'ROLB':'LB',
 'LB':'LB',
 'DB':'S',
 'DE':'DL',
 'NT':'DL',
 'RT':'OL',
 'LT': 'OL',
 'SE': 'WR',
 'FL': 'WR',
 'OLB':'LB',
 'DE ': 'DL',
 'OT': 'OL',
 'OL': 'OL',
 'LS': 'OL',
 'ILB':'LB',
 'G':'OL',
 'OG':'OL',
 'EDGE':'DL',
 'G,T': 'OL',
 'LS,TE': 'TE',
  'PR':'WR',
  'KR':'WR',
'RDE':'DL',
    'LDE':'DL',
        'RLB':'LB',
            'LLB':'LB',
    'UT':'LB',
    'PK':'K',
    'HB':'RB',
    'SB':'LB',
    'OB':'LB'

}

In [ ]:
import random
predicted_pos = []
for listed_p,model_p in zip(player_pos,pred_pos):
    #print(possible_pos)

    possible_pos = [_.upper() for _ in listed_p if _ != '0.0' and _ != 0]
    pposs = []
    for pos in list(possible_pos):
        if '/' in pos:
            pp = pos.split('/')
            for ppp in pp:
                pposs.append(ppp)
        else:
            pposs.append(pos)
    possible_pos = [pos_mapping[_.replace(' ','')] for _ in set(pposs)]
    #print(possible_pos,model_p)
    #print(listed_p)
    if model_p in possible_pos:
        true_p = model_p
        if listed_p[0] == 'DB':
            true_p =['S','CB'][int(random.random() > 0.5)]
            print(true_p)
    elif len(possible_pos) > 0:
        random.shuffle(possible_pos)
        true_p = possible_pos[0]
    else:
        true_p = model_p
    predicted_pos.append(true_p)
predicted_pos = np.array(predicted_pos)

In [ ]:
player_pos

In [ ]:
ratings = np.zeros((predicted_pos.shape[0],y.shape[1]))
for pos in unique_pos:

    Xn = np.nan_to_num(np.array(player_vectors))[np.where(predicted_pos == pos)]
    # tuned this to get roughly 8-12 players at 70 or above. Which seemed like normal for a league
    #scalerX2 = preprocessing.StandardScaler()#(quantile_range=(30.0, 70.0))
    scalerX2 = preprocessing.StandardScaler()

    Xn_s =scalerX2.fit_transform(np.nan_to_num(Xn))
    Xn_fs = fexps[pos].transform(np.nan_to_num(Xn_s))
    predict = clfs[pos].predict(Xn_fs)
    est_ratings = np.nan_to_num(scalersY[pos].inverse_transform(predict))
    ratings[np.where(predicted_pos == pos)] = est_ratings


In [ ]:
#all_tables[2018]['nwe'].keys()
#'logo', 'name', 'conf'
teams2 = []
for t in teams:
    try:
        aaaa = all_tables[GEN_YEAR][t]['conf']
        teams2.append(t)
    except:
        pass
teams = teams2

In [ ]:
confs = ['All']
divs = list(set([all_tables[GEN_YEAR][t]['conf'][1] for t in teams]))
divs

In [ ]:
all_tables[GEN_YEAR][t1]['conf']

In [ ]:
import random
base = {}

base['startingSeason'] = GEN_YEAR
valid_tids = list(range(len(teams)))

new_teams = []

for i,t1 in enumerate(sorted(list(teams))):
    t = {}
    t['abbrev'] = t1.upper()[:3]
    nsplit = all_tables[GEN_YEAR][t1]['name'].split(' ')
    t['region'] = ' '.join(nsplit[:-1])
    t['name'] = nsplit[-1]
    t['imgURL'] = all_tables[GEN_YEAR][t1]['logo']
    t['tid'] = i 
    div_name = all_tables[GEN_YEAR][t1]['conf'][1]
    t['did'] = divs.index(div_name)
    t['stadiumCapacity'] = 40000

    t['cid'] = 0#[c in all_tables[GEN_YEAR][t1]['conf'] for c in confs].index(True)
    new_teams.append(t)
print(new_teams)
team_abbbrevs = list(teams)
    
base['teams'] = new_teams
base['gameAttributes'] = []
base['version'] = 34
DIVS = []
for div in divs:
    cid = 0#[c in div for c in confs].index(True)
    did = divs.index(div)
    DIVS.append({'did':did, 'cid':cid, 'name':div})
print(DIVS)
base['gameAttributes'] = [{'key':'numGamesPlayoffSeries', 'value':[1 for i in range(3)]}]
base['gameAttributes'].append({'key':'divs', 'value': DIVS})
base['gameAttributes'].append({'key':'confs', 'value': [{'cid':i, 'name':n } for i,n in enumerate(confs)]})

base['gameAttributes'].append({'key':'numGames', 'value': 12})
base['gameAttributes'].append({"key": "maxContract","value": 7500})
base['gameAttributes'].append({"key": "maxRosterSize","value": 120})
base['gameAttributes'].append({'key':'numPlayoffByes', 'value': 0})


In [ ]:
base['gameAttributes']

In [ ]:
if gen_FA > 0:
    player_names_f = player_names + ["Free Agent{}".format(write_roman(i)) for i in range(gen_FA)]
    player_years_f = player_years + [GEN_YEAR for i in range(gen_FA)]
    # scale them down, we don't want a bunch of amazing replacements
    MEAN_S = 0.95
    STD_S = 0.25
    rp_ratings = np.random.multivariate_normal(MEAN_S*replacement_player_mean_r,STD_S*replacement_player_cov_r,size=(gen_FA))
    ratings_f = np.vstack([ratings,rp_ratings])
else:
    player_names_f = player_names
    player_years_f = player_years
    ratings_f = ratings

In [ ]:
l2 = [_.lower() for _ in y.columns]
l2[1] = 'stre'
l2[3] = 'endu'
l2[-7] = 'tck'

In [ ]:
import pprint
import copy
import random
gen_player = []
pp = pprint.PrettyPrinter()
for i,name in enumerate(player_names_f):
    py = player_years_f[i]
    name = player_names_f[i].replace('*','').replace('+','')
    if name in ['Team Total','Opp Total']:
        continue
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])
    
     
    r_vec = {k: ratings_f[i][i2] for i2,k in enumerate(l2)}
    r_vec = {k: int(np.clip(v,0,100)) for k,v in r_vec.items()}
    #for t in ['hgt', 'stre', 'spd', 'jmp', 'endu', 'ins', 'dnk', 'ft', 'fg', 'tp', 'diq', 'oiq', 'drb', 'pss', 'reb']:
    #    if not t in r_vec:
    #        r_vec[t] = 50

    true_pos = [predicted_pos[i]]
    r_vec['pos'] = true_pos[0]
    new_player['pos'] =true_pos[0]
    #print(set(true_pos))


    new_player['ratings'] = [r_vec]
    age = player_ages[i]
    new_player['born'] = {'year':GEN_YEAR-age,'loc':''}
    new_player['tid'] = team_abbbrevs.index(player_teams[i])

    exp = player_drafts[i]
    try:
        exp = int(exp)
    except:
        exp = 0
    new_player['draft'] = {"originalTid": -1, "pick": 0, "round": 0, "tid": -1, "year": GEN_YEAR-exp}
    new_player['weight'] = int(float(player_weights[i]))
    try:
        ht = [int(_) for _ in player_heights[i].split('-')]
        hgt = ht[0]*12 + ht[1]
        #print(ht,hgt)
        new_player['hgt'] = int(hgt)#int(3.7*(hgt-66))
    except:
        pass
    gen_player.append(new_player)


In [ ]:
#gen_player

In [ ]:
sum(['pos' in _['ratings'][0] for _ in gen_player]),len(gen_player)

In [ ]:
base['players'] = gen_player
with open('cfbm_roster_{}.json'.format(tyear[0]),'wt') as fp:
    json.dump(base,fp, sort_keys=True)

In [ ]:
len(ratings),len(ratings_f),len(ratings)+gen_FA
gen_player

In [ ]:
sorted(['S','DL'])

In [ ]:
ppos = [_.upper() for _ in set(sum(player_pos,[])) if _ != '0.0']

In [ ]:
pPos = []
for p in list(ppos):
    if '/' in p:
        for pp in p.split('/'):
            pPos.append(pp)
    else:
        pPos.append(p)
pPos = list(set(pPos))

In [ ]:
df.Pos.unique()